In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [ ]:
!wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" 

--2022-11-14 14:00:00--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.134.128, 173.194.210.128, 173.194.211.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.134.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘happy-or-sad.zip’

happy-or-sad.zip    100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2022-11-14 14:00:00 (151 MB/s) - ‘happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
import zipfile
local_zip= './happy-or-sad.zip'
zip_ref=zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./happy-or-sad')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
base_dir = os.path.join("./happy-or-sad")
happy_dir=os.path.join( base_dir, "happy/")
sad_dir=os.path.join(base_dir,"sad/")

In [ ]:
img=load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[3])}")

In [ ]:
arr=img_to_array(img)
print(arr.shape)
print(np.max(arr))

(150, 150, 3)
255.0


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data=ImageDataGenerator(rescale=1./255)
train_generator=train_data.flow_from_directory(base_dir, target_size=(150,150), batch_size=10, class_mode='binary')


Found 80 images belonging to 2 classes.


In [ ]:
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
                          tf.keras.layers.MaxPooling2D(2,2),
                          tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
                          tf.keras.layers.MaxPooling2D(2,2),
                          tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
                          tf.keras.layers.MaxPooling2D(2,2),
                          tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
                          tf.keras.layers.MaxPooling2D(2,2),
                          tf.keras.layers.Flatten(),
                          tf.keras.layers.Dense(128, activation='relu'),
                          tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>0.99):
      print("staopp")
      self.model.stop_training=True
callbacks=myCallbacks()

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=50,
      verbose=1)
    

Epoch 1/50
8/8 [==============================] - 2s 234ms/step - loss: 5.3958e-05 - accuracy: 1.0000
Epoch 2/50
8/8 [==============================] - 2s 234ms/step - loss: 3.3121e-05 - accuracy: 1.0000
Epoch 3/50
8/8 [==============================] - 2s 236ms/step - loss: 2.2655e-05 - accuracy: 1.0000
Epoch 4/50
8/8 [==============================] - 2s 236ms/step - loss: 1.5527e-05 - accuracy: 1.0000
Epoch 5/50
8/8 [==============================] - 2s 246ms/step - loss: 1.0301e-05 - accuracy: 1.0000
Epoch 6/50
8/8 [==============================] - 2s 232ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 7/50
8/8 [==============================] - 2s 233ms/step - loss: 2.6612e-05 - accuracy: 1.0000
Epoch 8/50
8/8 [==============================] - 2s 231ms/step - loss: 1.7722e-05 - accuracy: 1.0000
Epoch 9/50
8/8 [==============================] - 2s 232ms/step - loss: 1.4928e-05 - accuracy: 1.0000
Epoch 10/50
8/8 [==============================] - 2s 235ms/step - loss: 1.1805e-05 - 

In [ ]:
classification=model.predict(train_generator)
print(classification[3])

8/8 [==============================] - 1s 80ms/step
[5.5694555e-10]


In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is sad")
  else:
    print(fn + " is happy")

KeyboardInterrupt: ignored